<a href="https://colab.research.google.com/github/dtoralg/IE_Calidad_ML/blob/main/Ejercicios/Modulo%206/Modulo_6_Ejercicio_4_Bagging_vs_Boosting_Resuelto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Ejercicio 4: Bagging vs Boosting — Comparativa con Datos Ruidosos**
**Propósito**: Evaluar la robustez de modelos ensemble (Random Forest vs XGBoost) ante la presencia de ruido en datos industriales reales.

## Introducción

En este ejercicio evaluaremos la **robustez de técnicas de ensamblado (ensemble)** en presencia de datos ruidosos utilizando un dataset real del entorno de fabricación industrial. Compararemos dos enfoques:

- **Bagging** mediante `RandomForestClassifier`.
- **Boosting** mediante `XGBClassifier`.

Trabajaremos con el **SECOM Manufacturing Data Set**, que contiene mediciones de sensores con alta correlación espuria y ruido inherente.

Objetivos principales:

- Introducir **ruido artificial** y comparar el rendimiento de los modelos con y sin ruido.
- Evaluar métricas clave: **F1 Macro** y **recall por clase**.
- Discutir resultados en términos de **robustez** y **bias-variance tradeoff**.



> ### Recomendación:
> Las redes neuronales hacen millones de operaciones matemáticas (multiplicaciones de matrices, derivadas, etc.) durante el entrenamiento. Una CPU puede hacer estas operaciones, pero una GPU está diseñada específicamente para hacerlas en paralelo y mucho más rápido.
>
> Por ello te recomiendo que para estos ejercicios cambies tu entorno de ejecución en Colab.
>
> Puedes hacerlo haciendo click en `Entorno de Ejecución` en la barra superior, después `Cambiar tipo de entorno de ejecución` y selecciona `GPU`. Esto hará que tus notebook ejecuten más rápido los cálculos con Deep Learning.


## Carga de librerías y configuración del entorno

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import f1_score, recall_score
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
import requests
import zipfile
import io

# Estilo gráfico
sns.set(style='whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)

## Descripción del dataset: SECOM Manufacturing Data Set

**Fuente**: UCI Machine Learning Repository  
https://archive.ics.uci.edu/ml/datasets/SECOM

Este conjunto de datos fue recogido en un entorno real de fabricación de semiconductores y contiene:
- 1567 observaciones.
- 590 variables medidas por sensores.
- 1 variable objetivo binaria: `1` (producto defectuoso) / `-1` (producto correcto).

**Licencia del dataset:** SECOM Manufacturing Data Set — Fuente: UCI ML Repository, Licencia: [CC BY 4.0](https://creativecommons.org/licenses/by/4.0/)

## Carga y preparación de datos

In [2]:
zip_url = "https://github.com/dtoralg/IE_Calidad_ML/raw/refs/heads/main/Data/Modulo%206/secom.zip"

# Descargar el zip
response = requests.get(zip_url)

# Verificar que la descarga fue exitosa
if response.status_code == 200:
    # Leer el archivo ZIP en memoria
    zip_file = zipfile.ZipFile(io.BytesIO(response.content))

    # Mostrar el contenido del ZIP
    print("Archivos contenidos:")
    print(zip_file.namelist())

    # Extraer todos los archivos (opcional)
    zip_file.extractall("secom_data")  # Carpeta donde se guardarán
    print("Archivos extraídos en la carpeta 'secom_data'.")
else:
    print("Error al descargar el archivo:", response.status_code)

Archivos contenidos:
['secom.data', 'secom.names', 'secom_labels.data']
Archivos extraídos en la carpeta 'secom_data'.


In [3]:
# Cargar datos desde archivos locales
data_path = '/content/secom_data/secom.data'
labels_path = '/content/secom_data/secom_labels.data'

features_df = pd.read_csv(data_path, sep='\s+', header=None)
labels_df = pd.read_csv(labels_path, sep='\s+', header=None, names=["label", "timestamp"])

# Fusionar y preparar dataset
df = pd.concat([features_df, labels_df["label"]], axis=1)
df.dropna(axis=1, thresh=800, inplace=True)  # columnas con demasiados NaNs
df.fillna(df.median(numeric_only=True), inplace=True)  # imputación

# Separar variables y objetivo
X = df.drop(columns=["label"])
y = df["label"].replace({-1: 0, 1: 1})

# Escalar
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Dividir
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, stratify=y, random_state=42)

print("Distribución de clases en train:", y_train.value_counts().to_dict())

Distribución de clases en train: {0: 1023, 1: 73}


## Estrategias para tratar el desbalanceo de clases

La clase `1` (productos defectuosos) representa una proporción muy pequeña del dataset.  
Para mejorar el rendimiento del modelo en esta clase minoritaria, aplicaremos técnicas de balanceo:

- **SMOTE (Synthetic Minority Over-sampling Technique)**: genera ejemplos sintéticos de la clase minoritaria.
- **Ajuste de pesos en los modelos**: se le da mayor peso a la clase minoritaria para penalizar más sus errores.


In [4]:
# Aplicar SMOTE solo sobre el conjunto de entrenamiento
smote = SMOTE(random_state=42)
X_train_balanced, y_train_balanced = smote.fit_resample(X_train, y_train)

# También aplicamos ruido al conjunto balanceado para identificar que modelo se comporta mejor
np.random.seed(42)
noise_bal = np.random.normal(loc=0, scale=0.5, size=X_train_balanced.shape)
X_train_balanced_noisy = X_train_balanced + noise_bal

# Mostrar nueva distribución
print("Distribución tras SMOTE:", pd.Series(y_train_balanced).value_counts().to_dict())

Distribución tras SMOTE: {0: 1023, 1: 1023}


## Reentrenamiento con datos balanceados (SMOTE)

In [5]:
# Random Forest con SMOTE
rf_smote = RandomForestClassifier(random_state=42)
rf_smote.fit(X_train_balanced, y_train_balanced)
y_pred_rf_smote = rf_smote.predict(X_test)

# XGBoost con SMOTE
xgb_smote = XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss')
xgb_smote.fit(X_train_balanced, y_train_balanced)
y_pred_xgb_smote = xgb_smote.predict(X_test)

# Métricas
f1_rf_smote = f1_score(y_test, y_pred_rf_smote, average='macro')
f1_xgb_smote = f1_score(y_test, y_pred_xgb_smote, average='macro')
recall_rf_smote = recall_score(y_test, y_pred_rf_smote, average=None)
recall_xgb_smote = recall_score(y_test, y_pred_xgb_smote, average=None)

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [14:17:01] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


In [6]:
# Random Forest con SMOTE + ruido
rf_smote_noisy = RandomForestClassifier(random_state=42)
rf_smote_noisy.fit(X_train_balanced_noisy, y_train_balanced)
y_pred_rf_smote_noisy = rf_smote_noisy.predict(X_test)

# XGBoost con SMOTE + ruido
xgb_smote_noisy = XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss')
xgb_smote_noisy.fit(X_train_balanced_noisy, y_train_balanced)
y_pred_xgb_smote_noisy = xgb_smote_noisy.predict(X_test)

# Métricas
f1_rf_smote_noisy = f1_score(y_test, y_pred_rf_smote_noisy, average='macro')
f1_xgb_smote_noisy = f1_score(y_test, y_pred_xgb_smote_noisy, average='macro')
recall_rf_smote_noisy = recall_score(y_test, y_pred_rf_smote_noisy, average=None)
recall_xgb_smote_noisy = recall_score(y_test, y_pred_xgb_smote_noisy, average=None)

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [14:17:10] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


## Comparación de resultados con técnicas de balanceo

In [7]:
# Añadir nuevos resultados
results_df_bal = pd.DataFrame({
    "Modelo": [
        "Random Forest (SMOTE)", "XGBoost (SMOTE)",
        "Random Forest (SMOTE + Ruido)", "XGBoost (SMOTE + Ruido)"
    ],
    "F1 Macro": [f1_rf_smote, f1_xgb_smote, f1_rf_smote_noisy, f1_xgb_smote_noisy],
    "Recall Clase 0": [recall_rf_smote[0], recall_xgb_smote[0], recall_rf_smote_noisy[0], recall_xgb_smote_noisy[0]],
    "Recall Clase 1": [recall_rf_smote[1], recall_xgb_smote[1], recall_rf_smote_noisy[1], recall_xgb_smote_noisy[1]]
})


results_df_bal

,Modelo,F1 Macro,Recall Clase 0,Recall Clase 1
0,Random Forest (SMOTE),0.482986,1.000000,0.000000
1,XGBoost (SMOTE),0.563954,0.993182,0.096774
2,Random Forest (SMOTE + Ruido),0.574526,0.906818,0.290323
3,XGBoost (SMOTE + Ruido),0.624042,0.931818,0.354839


## Conclusiones

- **XGBoost** muestra mayor robustez ante ruido.
- **XGBoost** tiene mejor F1 al incluir cierto ruido ya que consigue generalizar bien al introducirlo, en una medida similar a RandomForest
- El **recall para la clase minoritaria** mejora considerablemente con Boosting + ruido. Esto se debe a que el modelo original tiende a sobreajustarse, aun con el uso de SMOTE, y con el ruido introducido pierde capacidad predictiva de la clase 0 en favor de la clase 1 (generaliza mejor)

Este ejercicio demuestra el **trade-off entre bias y varianza**:
- Bagging reduce varianza, es más robusto.
- Boosting reduce sesgo, pero es más sensible al ruido.

## Próximos pasos

- Ajustar hiperparámetros para cada modelo.
- Aplicar técnicas de selección de características para reducir dimensionalidad.
- Explorar otras técnicas de ensemble como Stacking.

---

Dataset original: SECOM Manufacturing Data Set — UCI Machine Learning Repository
